# 主成分分析 Principal Components Analysis

在讨论因子分析时，我们将数据 $x \in \mathbb{R}^n$ 建模为近似地处于较低的 $k$ 维空间中，$k \ll m$。具体来说，我们假设每个点 $x^{(i)}$ 的生成过程，包含两个步骤，首先在 $k$ 维远交空间 $\{ \Lambda z + \mu; z \in \mathbb{R}^k \}$ 生成点 $z^{(i)}$，然后再增加协方差 $\Psi$ 噪声。因子分析基于概率模型，然后使用迭代的EM算法来进行参数估计。

而本节介绍的主成分分析，同样致力于将数据近似地降低到低维空间中。但是主成分分析更直接，仅需要计算特征向量，并不依赖于EM算法。

假设目前给定训练集 $\{x^{(i)};i=1,\cdots,m\}$ 包含 $m$ 中不同类型汽车的各类属性，比如最高时速，转弯半径等等，而 $n \ll m$。但我们所不知道的是，有两个属性 $x_i$ 和 $x_j$ 分别表示汽车按英里数计的最高时速，和按公里计的最高时速。这两个属性几乎线性相关，取决于四舍五入后所引入的微小差异。所以，数据实际上存在于 $n-1$ 的低维空间中。如何能自动地检测到并移除这部分冗余呢？

再来看一个更自然的例子，考虑一个关于遥控直升机的问卷调查。$x_1^{(i)}$ 用来测量用户 $i$ 的操作水平，而 $x_2^{(i)}$ 用来测量用户是否喜欢操作遥控直升机。由于遥控直升机非常难操作，只有那些特别投入的人，特别喜欢遥控直升机的人，才更可能操作得好。所以 $x_1$ 和 $x_2$ 是高度相关的属性。事实上，如果绘制一张散点图，数据点可能会落在某条直线附近，而上下浮动是由另一个很小并与之正交的噪声变量导致的。如何能自动地检测到这条直线的方向呢？

接下来会介绍主成分分析法。但在主成分分析法之前，通常需要进行数据清洗，根据均值和方差进行标准化：
1. 令 $\mu = \frac{1}{m}\sum_{i=1}^m x^{(i)}$
2. 将所有的 $x^{(i)}$ 替换为 $x^{(i)}-\mu$
3. 令 $\sigma_j^2 = \frac{1}{m}\sum_i(x_j^{(i)})^2$
4. 讲所有的 $x_j^{(i)}$ 替换为 $x_j^{(i)}/\sigma_j$

前两步使得数据的均值为零，后两步根据单位方差调整数据的范围，保证不同的属性（特征）的量纲相同，从而使得不同的属性之间可比。如果有先验知识，知道各个指标已经处于可比的状态，那么步骤3和步骤4也可以省略，比如数据项表示一张图片各个像素点的灰度值，灰度值本身在不同像素间已经是可比的。

完成标准化的操作之后，则需要计算最能体现数据差异的坐标向量 $\mu$。一个简易的思路是：找到单位向量 $\mu$ 使得所有数据点投影到该向量上的一系列值方差最大。直觉上看，所有数据都初始携带了一定量的方差/信息，我们希望找到向量 $\mu$ 使得数据在这个方向/次平面上，尽可能保留多的原始方差。